In [16]:
import pandas as pd

path = r'D:\Data Deep Learning\FINAL-DATASET\FINAL-DATASET\result.csv'

df = pd.read_csv(path)

df.head()

,imagename,answer,prob,prob_0,prob_1,prob_2,prob_3,prob_4
0,folder1_image_5865.jpg,4,0.999708,0.000129,0.000157,0.000005,1.801262e-08,0.999708
1,folder1_image_1277.jpg,4,0.999708,0.000118,0.000171,0.000003,5.518995e-08,0.999708
2,folder2_image_592.jpg,4,0.999708,0.000108,0.000178,0.000006,7.814338e-09,0.999708
3,folder1_image_25904.jpg,4,0.999704,0.000107,0.000186,0.000002,6.559484e-09,0.999704
4,folder1_image_29700.jpg,4,0.999633,0.000104,0.000258,0.000005,1.411263e-08,0.999633


In [17]:
df['answer'].value_counts()

answer
4    34853
1    31192
0    24481
2     1263
3       55
Name: count, dtype: int64

In [27]:
# Converting class > 2 to 2

df.loc[df['answer'] > 2, 'answer'] = 2



In [28]:
df['answer'].value_counts()

answer
2    36171
1    31192
0    24481
Name: count, dtype: int64

In [29]:
from sklearn.utils import class_weight

class_weights = dict(zip(
    df['answer'].unique(),
    class_weight.compute_class_weight(
        class_weight='balanced', 
        classes=df['answer'].unique(), 
        y=df['answer'])
))

class_weights

{2: 0.8463870688304627, 1: 0.9814909805933145, 0: 1.250548044061381}